In [7]:
import os
import pandas as pd
import archook

# Use archook to import arcpy
archook.get_arcpy()
import arcpy
from arcpy.sa import *

# Uplope Calculations for Bike Assignment

### Define Input Locations

In [11]:
# Emme network shapefile location
emme_shapefile = r'R:\SoundCast\Inputs\2014\networks\edges_0.shp'

output_dir = r'R:\Bike\slope\soundcast'

# Create a geodatabase for working with data
geodb = r'soundcast_slope.gdb'
arcpy.env.workspace = os.path.join(output_dir, geodb)

# name of Emme network shapefile, after exported to geodatabase
in_fc = geodb + r'\edges_0'

# Elevation raster location
in_raster = r'W:\geodata\raster\dem30m'

### Load and Process Data

In [5]:
# Create new geodatabase
arcpy.CreateFileGDB_management(output_dir, geodb)

# Add soundcast network shapefile to feature class
arcpy.FeatureClassToGeodatabase_conversion(emme_shapefile, os.path.join(output_dir,geodb))

<Result 'R:\\Bike\\slope\\soundcast\\soundcast_slope.gdb'>

In [76]:
arcpy.env.workspace = os.path.join(output_dir,geodb)

# Add field to the feature class, concatentating the inode and jnode for a unique ID
arcpy.AddField_management('edges_0', 'ID', "TEXT")

arcpy.CalculateField_management('edges_0', 'ID', "str(!NewINode!)+'-'+str(!NewJNode!)",'PYTHON_9.3')

<Result 'R:\\Bike\\slope\\soundcast\\soundcast_slope.gdb\\edges_0'>

In [91]:
# Find two-way links - we only need to split these links once
emme_links = arcpy.da.FeatureClassToNumPyArray(os.path.join(output_dir,in_fc), ('ID','Shape_Length','NewINode','NewJNode'))
df = pd.DataFrame(emme_links)

In [92]:
# Change geodatabase field names to match more generic Emme shapefile network output format

# Convert shape_length to miles
df['LENGTH'] = df['Shape_Length']/5280
df.rename(columns={'NewINode': 'INODE', 'NewJNode': 'JNODE'}, inplace=True)
df.drop('Shape_Length',axis=1,inplace=True)

In [39]:
# Compare this network versus existing slope attributes
# Only process on the subset that don't have slope already attached
old_attr = pd.read_csv(r'R:\SoundCast\Inputs\2014\bikes\emme_attr.in',sep=' ')
old_attr['ID'] = old_attr['inode'].astype('str')+'-'+old_attr['jnode'].astype('str')

# List of link IDs from imported network not in current attr file
len(df[-df['ID'].isin(old_attr['ID'].values)])

18449

In [50]:
# loop through each link 
ij_links = []
ji_links = []

for rownum in xrange(len(df)):
    inode = df.iloc[rownum].INODE
    jnode = df.iloc[rownum].JNODE
    ij_df = df[(df['INODE']==inode)&(df['JNODE']==jnode)]
    if len(ij_df) == 1:
        ij_id = ij_df.ID.values[0]
    
    ji_df = df[(df['INODE']==jnode)&(df['JNODE']==inode)]
    if len(ji_df) == 1:
        ji_id = ji_df.ID.values[0]
    else:
        # indicates a one-way link with no ji
        # append to ij_links and skip to next
        ij_links.append(ij_id)
        continue
    
    if ji_id not in ij_links:
        ij_links.append(ij_id)
    else:
        ji_links.append(ij_id)

In [51]:
ij_df = df[df['ID'].isin(ij_links)]
ji_df = df[df['ID'].isin(ji_links)]

In [52]:
len(ij_df)+len(ji_df)==len(df)

True

- export results to feature class

- split network lines into points
- note: this takes 30-60 minutes

- should only have to do this one time
- subsequent buffering will compare the imported shapefile links to the result of the cold-start process and only work on the links that haven't been processed yet, appending them to the existing results

In [94]:
# Loop through each network link and split the line into points, saving them in points array
points = []
sr = arcpy.Describe(os.path.join(output_dir,in_fc)).spatialReference
counter = 0
id_field = 'ID'   

# Set the step based on the line's length
# How many times must the line be split
# 
# Using 30 of fidelity - data is available every 30 meters so this may be unnecessary
segment_len = 30

# Create seperate array that holds tuple of link ID and point coordinates (in state plane coords)
final_result = []
count = 0
with arcpy.da.SearchCursor(os.path.join(output_dir,in_fc),["SHAPE@",id_field], spatial_reference=sr) as cursor:  
    for row in cursor:
        # Only process IJ links, because JI are exactly the same polyline shape
        if row[1] in ij_links:
            count += 1
            split_count = int(row[0].length/segment_len)
            big_output = []
            for i in range(split_count):
                point = row[0].positionAlongLine(i*segment_len)
                points.append(point)
                x = point.firstPoint.X
                y = point.firstPoint.Y
                point_list = []
                point_list.append(x)
                point_list.append(y)
                output = (str(row[1]), (x, y))
                final_result.append(output)

- Export results to a feature class called link_components

In [95]:
point_out_new = r'link_components'
# arcpy.CopyFeatures_management(points, r"R:\Bike\slope\bkr\bkr_sample.gdb" + point_out_new)
# delete_fc = True
# if delete_fc:
#     arcpy.DeleteFeatures_management(r"R:\Bike\slope\bkr\bkr_network.gdb" + point_out_new)
arcpy.CopyFeatures_management(points, os.path.join(output_dir,geodb,point_out_new))

<Result 'R:\\Bike\\slope\\soundcast\\soundcast_slope.gdb\\link_components'>

In [96]:
os.path.join(output_dir,geodb,point_out_new)

'R:\\Bike\\slope\\soundcast\\soundcast_slope.gdb\\link_components'

- Intersect the points with the links to get the edge IDs

In [98]:
inFeatures = ["link_components", "edges_0"]
intersectOutput = "link_components_full"
clusterTolerance = 1.5    
arcpy.Intersect_analysis(inFeatures, intersectOutput, "", clusterTolerance, "point")

<Result 'R:\\Bike\\slope\\soundcast\\soundcast_slope.gdb\\link_components_full'>

- Intersect with a raster to get elevation
- import elevation raster from W:/geodata/raster/dem30m
    - this is the raster of elevations at 30 m fidelity
    - note that elevation values are in METERS

In [99]:
# Note that spatial analyst must be active for this portion
arcpy.CheckOutExtension("Spatial")
# arcpy.env.workspace = r'R:\Bike\slope\bkr\bkr_network.gdb'

in_point_features = r'link_components_full'
out_point_features = r'link_components_elevation'
ExtractValuesToPoints(in_point_features, in_raster, out_point_features)

<geoprocessing server result object at 0x173b08d0>

In [102]:
# Read resulting intersection of points with elevation into numpy/pandas
elevation_shp = arcpy.da.FeatureClassToNumPyArray(out_point_features, ('RASTERVALU','ID','NewINode','NewJNode'))
df = pd.DataFrame(elevation_shp)

In [103]:
# List of links IDs
link_list = df.groupby('ID').min().index

In [104]:
# Loop through all edges
# Assume that all links are bi-directional and compute ij and ji direction slopes
# if a line is truly one-way, we will discard the ji direction
# since most are two-way it's worth it calculate for all links and merge results later
upslope_ij = {}
upslope_ji = {}
for link in link_list: 
    link_df = df[df['ID'] == link]

    # Extract the elevation data to numPy because it's faster to loop over
    elev_data = link_df['RASTERVALU'].values

    # Loop through each point in each edge
    upslope_ij[link] = 0
    upslope_ji[link] = 0
    for point in xrange(len(elev_data)-1):  # stop short of the list because we only want to compare the 2nd to last to last
        elev_diff = elev_data[point+1] - elev_data[point]
        if elev_diff > 0:
            upslope_ij[link] += elev_diff
        elif elev_diff < 0:
            upslope_ji[link] += abs(elev_diff)      # since we know it will be "negative" for the JI direction when calculated
                                                    # in references to the IJ direction

In [106]:
# Import dictionary to a series and attach upslope back on the original dataframe
upslope_ij_s = pd.Series(upslope_ij, name='elev_gain_ij')
upslope_ji_s = pd.Series(upslope_ji, name='elev_gain_ji')
upslope_ij_s.index.name='ID'
upslope_ij_s = upslope_ij_s.reset_index()
upslope_ji_s.index.name='ID'
upslope_ji_s = upslope_ji_s.reset_index()

# Attach ij-direction slope to IJ links
slope_ij = pd.merge(ij_df,upslope_ij_s,on='ID')
slope_ij.rename(columns={"elev_gain_ij": "elev_gain"}, inplace=True)

# Attach ji-direction slope to JI links

# fo JI links, flip the i and j values to get lookup of ji links
upslope_ji_s['newID'] = upslope_ji_s.ID.apply(lambda row: row.split('-')[-1]+"-"+row.split('-')[0])
slope_ji = pd.merge(ji_df,upslope_ji_s,left_on='ID',right_on='newID')
slope_ji.rename(columns={"elev_gain_ji": "elev_gain"}, inplace=True)
slope_ji['ID'] = slope_ji['newID']
slope_ji.drop(['ID_x','ID_y','newID'],axis=1,inplace=True)

# Append ji rows to ij to get a complete list of links
slope_df = slope_ij.append(slope_ji)

# Convert elevation into feet from meters
slope_df['elev_gain'] = slope_df['elev_gain']*3.28084

In [107]:
# Calcualte the average upslope in feet/feet
# Network distance measured in: miles, elevation in meters 
slope_df['avg_upslope'] = slope_df['elev_gain']/(slope_df['LENGTH']*5280)

- reformat and export as emme_attr.in
- for BKR, assume all bike facilities are 0 for now

In [109]:
emme_attr = slope_df
emme_attr.rename(columns={'INODE':'inode','JNODE':'jnode','avg_upslope':'@upslp'},
                inplace=True)

emme_attr.drop(['LENGTH','elev_gain'], axis=1, inplace=True)

# add bike facility of 0
emme_attr['@bkfac'] = 0

- some very short links are not processed
- assume zero elevation change for these

In [113]:
# Get list of IDs from network not included in the final outpu
df = pd.DataFrame(emme_links)

In [114]:
print len(emme_attr)
print len(df)

34224
34271


In [119]:
missing_links = df[-df['ID'].isin(emme_attr['ID'].values)]
missing_links = missing_links[['ID','NewINode','NewJNode']]
missing_links.columns = [i.lower() for i in missing_links.columns] 

missing_links['@upslp'] = 0
missing_links['@bkfac'] = 0

emme_attr = emme_attr.append(missing_links)

emme_attr.drop(['id','ID'], axis=1,inplace=True)
emme_attr = emme_attr[['inode','jnode','@bkfac','@upslp']]

In [120]:
# Export emme transaction file
emme_attr.to_csv(output_dir + r'\emme_attr.in', sep=' ', index=False)

# Export version for use in ArcMap
emme_attr['id']=emme_attr['inode'].astype('str')+'-'+emme_attr['jnode'].astype('str')
emme_attr.to_csv(output_dir + r'\emme_attr.csv', sep=' ', index=False)

In [121]:
# Load the soundcast results for comparison
df = pd.read_csv(r'R:\SoundCast\Inputs\2014\bikes\emme_attr.in', sep=' ')

In [122]:
print df['@upslp'].max()
print df['@upslp'].mean()
print df['@upslp'].median()

22.3756189894
0.0332534048304
0.0192160023467


In [123]:
print emme_attr['@upslp'].max()
print emme_attr['@upslp'].mean()
print emme_attr['@upslp'].median()

7.41235999936
0.0256037075541
0.0102209055315


In [124]:
emme_attr

,inode,jnode,@bkfac,@upslp,id
0,148702,148696,0,0.000000,148702.0-148696.0
1,69493,69494,0,0.000000,69493.0-69494.0
2,119584,119589,0,0.003672,119584.0-119589.0
3,119589,119598,0,0.005043,119589.0-119598.0
4,119598,119606,0,0.015402,119598.0-119606.0
5,153482,153511,0,0.031945,153482.0-153511.0
6,123525,123529,0,0.046859,123525.0-123529.0
7,71301,71533,0,0.000000,71301.0-71533.0
8,71533,71639,0,0.000000,71533.0-71639.0
9,68363,68348,0,0.000000,68363.0-68348.0
